In [1]:
import os
%pwd
os.chdir('../')
%pwd

'C:\\Users\\Franklin NDJATHE\\Documents\\projects\\End-to-End-ML-Project-with-MLFlow'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ndjathe-ndjathe/End-to-End-ML-Project-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ndjathe-ndjathe"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b35f96a5e048fdb09da658cd5dcc2f3c37a8d2f2"

In [14]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [15]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [18]:
from typing import ClassVar
from box import ConfigBox

@dataclass
class ConfigurationManager:
    config_filepath: Path = CONFIG_FILE_PATH
    params_filepath: Path = PARAMS_FILE_PATH
    schema_filepath: Path = SCHEMA_FILE_PATH
    
    config: ClassVar[ConfigBox] = read_yaml(config_filepath)
    params: ClassVar[ConfigBox] = read_yaml(params_filepath)
    schema: ClassVar[ConfigBox] = read_yaml(schema_filepath)
    
    def __post_init__(self):
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"]
        )
        
        return model_evaluation_config

[2023-07-29 21:31:20,373: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-29 21:31:20,375: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 21:31:20,376: INFO: common: yaml file: schema.yaml loaded successfully]


In [19]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [20]:
@dataclass
class ModelEvaluation:
    config: ModelEvaluationConfig
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # Please refer to the doc for more information:
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-07-29 21:31:31,675: INFO: common: created directory at: artifacts]
[2023-07-29 21:31:31,676: INFO: common: created directory at: artifacts/model_evaluation]
[2023-07-29 21:31:32,052: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/07/29 21:31:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
